The notebook uses the preprocessing from From : https://www.kaggle.com/code/takanashihumbert/magic-bingo-train-part-lb-0-687
and from https://www.kaggle.com/code/leehomhuang/catboost-baseline-with-lots-features-inference
with some new features.

And ideas from https://www.kaggle.com/code/cdeotte/xgboost-baseline-0-676 as well.

In [1]:

import os 
os.chdir("../")

In [2]:
!pip install /kaggle/input/polars-for-student/polars-0.16.9-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/polars-for-student/typing_extensions-4.5.0-py3-none-any.whl

Processing ./input/polars-for-student/polars-0.16.9-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/polars-for-student/polars-0.16.9-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'

Processing ./input/polars-for-student/typing_extensions-4.5.0-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/polars-for-student/typing_extensions-4.5.0-py3-none-any.whl'



In [3]:
import gc
import os
from input.utilcode.utils.constants import *
import pandas as pd
import numpy as np
import warnings
import pickle
import polars as pl

from collections import defaultdict
from itertools import combinations
import pyarrow as pa

from xgboost import XGBClassifier
from tsfresh.feature_extraction import feature_calculators

from lightgbm import LGBMClassifier
from lightgbm import early_stopping
from lightgbm import log_evaluation

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score


import matplotlib.pyplot as plt
from colorama import Fore, Back, Style

{1: '0-4', 2: '0-4', 3: '0-4', 4: '5-12', 6: '5-12', 7: '5-12', 9: '5-12', 11: '5-12', 12: '5-12', 5: '5-12_low', 8: '5-12_low', 10: '5-12_low', 13: '5-12_low', 14: '13-22', 16: '13-22', 17: '13-22', 15: '13-22_low', 18: '13-22_high'}
['undefined', '\\u00f0\\u0178\\u02dc\\u0090', 'Hey!', 'I should go to the Capitol and tell Mrs. M!', 'This is perfect for the exhibit.', 'Look at all those activists!', "If I were you, I'd go to the library and do some digging.", 'Ugh. Fine.', '\\u00f0\\u0178\\u02dc\\u00b4', 'You can talk to a textile expert there.', "Hey Jo, let's take a look at the shirt!", "Hang on. I'll get you out of there!", 'Teddy!!!', 'I wonder whose glasses these are.', 'Can you help me tidy up?', 'Thanks for your help, kid!', 'Your grampa is waiting for you in the collection room.', 'Thanks for the help!', "Hmmm... not sure. Why don't you try the library?", 'Do you know who Theodora Youmans is?', 'I should go to the Capitol and tell everyone!', "And look! She's wearing the shirt

In [4]:
i_features={12:{'*COUGH COUGH COUGH*_text_ET_max_',
 'Can you help me tidy up?_text_ET_max_',
 "Check out our microfiche. It's right through that door._text_ET_max_",
 "Check out our microfiche. It's right through that door._text_ET_sum_",
 'Do you have any info on Theodora Youmans?_text_ET_max_',
 'Do you have any info on Theodora Youmans?_text_ET_sum_',
 'Do you know anything about this slip?_text_ET_sum_',
 'Ha! Good one._text_ET_sum_',
 'He was looking for a taxidermist._text_ET_max_',
 "He's always trying to get you in trouble, and he doesn't like animals!_text_ET_max_",
 'Hello there!_text_ET_max_',
 'Hey, this is Youmans!_text_ET_sum_',
 "Hmm. Button's still not working._text_ET_sum_",
 'How can I help you?_text_ET_max_',
 "I don't know!_text_ET_max_",
 "I don't need that right now._text_ET_max_",
 'I got here and the whole place was ransacked!_text_ET_max_',
 'I got that one from my Gramps!_text_ET_max_',
 "I haven't quite figured it out just yet..._text_ET_max_",
 'I knew I could count on you, Jo!_text_ET_sum_',
 'I should go to the Capitol and tell everyone!_text_ET_max_',
 "I'm Leopold's grandkid!_text_ET_max_",
 "I'm Leopold's grandkid!_text_ET_sum_",
 "I'm afraid my papers have gone missing in this mess._text_ET_max_",
 "I'm afraid my papers have gone missing in this mess._text_ET_sum_",
 "I'm sure you'll find Theodora in there somewhere!_text_counts",
 "I've got a stack of business cards from my favorite cleaners._text_ET_sum_",
 'Is this your coffee, Gramps?_text_ET_max_',
 "It's for Grampa Leo. He's a historian!_text_ET_max_",
 "It's for Grampa Leo. He's a historian!_text_ET_sum_",
 'Maybe I can help!_text_ET_max_',
 "Oh, that's from Bean Town._text_ET_max_",
 "Poor Gramps! I should make sure he's okay._text_ET_max_",
 'Right outside the door._text_ET_sum_',
 'Slow down, Jo._text_ET_max_',
 "Sorry, I'm too busy for kids right now._text_ET_sum_",
 "Sorry, can't help you._text_ET_max_",
 "Sorry, can't help you._text_ET_sum_",
 'Take a look!_text_ET_max_',
 'Thanks to them, Wisconsin was the first state to approve votes for women!_text_ET_max_',
 'Thanks to them, Wisconsin was the first state to approve votes for women!_text_ET_sum_',
 'The archivist said I should look in the stacks._text_ET_sum_',
 'Theodora Youmans must be the owner!_text_ET_max_',
 'Theodora Youmans must be the owner!_text_ET_sum_',
 'Theodora Youmans? Is that who owned the shirt?_text_ET_max_',
 'Theodora Youmans? Of course!_text_ET_max_',
 'They study clothes and fabric._text_ET_max_',
 'They study clothes and fabric._text_ET_sum_',
 'This button never works!_text_ET_max_',
 'Two missions, actually!_text_ET_sum_',
 "Wait a sec. Women couldn't vote?!_text_ET_max_",
 'Wells! What was he doing here? I should ask the librarian._text_ET_sum_',
 'Wells? I knew it!_text_ET_max_',
 'Wells? I knew it!_text_ET_sum_',
 'What happened here?!_text_ET_sum_',
 "What's a taxidermist?_text_ET_sum_",
 'Where are the Stacks?_text_ET_max_',
 'Where should I go again?_text_ET_sum_',
 "Who could've done this?_text_ET_max_",
 "Who could've done this?_text_ET_sum_",
 "Why don't you take a look?_text_ET_max_",
 'Wow!_text_ET_max_',
 'Yeah. Thanks anyway._text_ET_sum_',
 'Yep._text_ET_max_',
 'Yes! I was wondering-_text_ET_sum_',
 'Yes!_text_ET_sum_',
 'You could ask the archivist. He knows everybody!_text_ET_max_',
 'You could ask the archivist. He knows everybody!_text_ET_sum_',
 "You'll have to get started without me._text_ET_sum_",
 "You're still here? I'm trying to work!_text_counts",
 'Youmans was a suffragist here in Wisconsin._text_ET_max_',
 'Youmans was a suffragist!_text_ET_max_',
 'Your gramps is awesome! Always full of stories._text_ET_max_',
 '\\Taxidermy: the art of preparing, stuffing, and mounting the skins of animals.\\_text_ET_max_',
 '\\Taxidermy: the art of preparing, stuffing, and mounting the skins of animals.\\_text_ET_sum_',
 'block_ET_max_',
 'boss_ET_max_',
 'businesscards.card_0.next_ET_sum_',
 'businesscards.card_1.next_ET_max_',
 'businesscards.card_1.next_ET_mean_',
 'businesscards.card_bingo.next_ET_max_',
 'businesscards.card_bingo.next_ET_mean_',
 'businesscards.card_bingo.next_ET_sum_',
 'businesscards_ET_max_',
 'businesscards_ET_sum_',
 'checkpoint_ET_max_',
 'close_ET_max_',
 'cutscene_click_ET_max_',
 'directory_ET_max_',
 'door_block_talk_ET_max_',
 'door_block_talk_ET_mean_',
 'door_block_talk_ET_sum_',
 'gramps_ET_sum_',
 'gramps_fqid_counts',
 'hover_duration_max_',
 'hover_duration_mean_',
 'journals.hub.topics_ET_max_',
 'journals.hub.topics_ET_mean_',
 'journals.pic_1.next_ET_max_',
 'journals.pic_1.next_fqid_counts',
 'journals.pic_2.bingo_ET_max_',
 'journals.pic_2.bingo_ET_sum_',
 'journals.pic_2.next_ET_max_',
 'journals_ET_sum_',
 'journals_fqid_counts',
 'logbook.page.bingo_ET_max_',
 'logbook.page.bingo_ET_mean_',
 'logbook.page.bingo_ET_sum_',
 'logbook_ET_sum_',
 'logbook_fqid_counts',
 'map_click_ET_mean_x',
 'map_click_event_name_counts',
 'map_hover_ET_mean_',
 'map_hover_event_name_counts',
 'navigate_click_ET_max_x',
 'navigate_click_ET_mean_x',
 'notebook_click_ET_max_',
 'notebook_click_ET_max_x',
 'notification_click_ET_max_',
 'notification_click_ET_mean_',
 'object_click_ET_max_x',
 'object_click_event_name_counts',
 'outtolunch_ET_mean_',
 'page_mean_',
 'reader.paper0.prev_ET_max_',
 'reader.paper0.prev_ET_sum_',
 'reader.paper1.next_ET_max_',
 'reader.paper1.next_ET_mean_',
 'reader.paper1.prev_ET_max_',
 'reader.paper2.bingo_ET_sum_',
 'reader.paper2.next_ET_mean_',
 'reader_ET_sum_',
 'room_coor_x_mean_',
 'session_number_',
 'tobasement_ET_sum_',
 'tobasement_fqid_counts',
 'tocloset_dirty_ET_max_',
 'tocloset_dirty_ET_sum_',
 'toentry_ET_max_',
 'toentry_ET_sum_',
 'tofrontdesk_ET_mean_',
 'tohallway_ET_sum_',
 'tomap_ET_sum_',
 'tostacks_ET_mean_',
 'tostacks_ET_sum_',
 'tostacks_fqid_counts',
 'tunic.capitol_0.hall.boss.talktogramps_ET_mean_',
 'tunic.capitol_0.hall_ET_mean_',
 'tunic.capitol_0_ET_sum_',
 'tunic.capitol_1.hall_ET_sum_',
 'tunic.capitol_1_ET_max_',
 'tunic.capitol_1_ET_sum_',
 'tunic.drycleaner.frontdesk.logbook.page.bingo_ET_mean_',
 'tunic.drycleaner.frontdesk.worker.done_text_fqid_counts',
 'tunic.drycleaner.frontdesk.worker.hub_ET_mean_',
 'tunic.drycleaner.frontdesk_ET_mean_',
 'tunic.drycleaner.frontdesk_room_fqid_counts',
 'tunic.drycleaner_ET_sum_',
 'tunic.historicalsociety.closet_dirty.door_block_talk_ET_mean_',
 'tunic.historicalsociety.closet_dirty.door_block_talk_ET_sum_',
 'tunic.historicalsociety.closet_dirty.gramps.archivist_ET_mean_',
 'tunic.historicalsociety.closet_dirty.gramps.archivist_ET_sum_',
 'tunic.historicalsociety.closet_dirty.gramps.news_ET_max_',
 'tunic.historicalsociety.closet_dirty.gramps.news_ET_mean_',
 'tunic.historicalsociety.closet_dirty.gramps.news_ET_sum_',
 'tunic.historicalsociety.closet_dirty.trigger_scarf_ET_max_',
 'tunic.historicalsociety.collection_ET_max_',
 'tunic.historicalsociety.entry_ET_max_',
 'tunic.historicalsociety.entry_ET_mean_',
 'tunic.historicalsociety.frontdesk.archivist.have_glass_recap_ET_sum_',
 'tunic.historicalsociety.frontdesk.archivist.hello_ET_sum_',
 'tunic.historicalsociety.frontdesk.archivist.newspaper_ET_max_',
 'tunic.historicalsociety.frontdesk_ET_mean_',
 'tunic.historicalsociety.stacks_room_fqid_counts',
 'tunic.historicalsociety_ET_max_',
 'tunic.historicalsociety_ET_mean_',
 'tunic.hub.slip_ET_mean_',
 'tunic.humanecology.frontdesk.worker.badger_ET_max_',
 'tunic.humanecology.frontdesk_room_fqid_counts',
 'tunic.humanecology_ET_max_',
 'tunic.humanecology_ET_mean_',
 'tunic.humanecology_ET_sum_',
 'tunic.kohlcenter.halloffame_ET_mean_',
 'tunic.kohlcenter_ET_mean_',
 'tunic.kohlcenter_fqid_counts',
 'tunic.library.frontdesk.worker.wells_ET_max_',
 'tunic.library.frontdesk.worker.wells_recap_text_fqid_counts',
 'tunic.library.frontdesk_ET_max_',
 'tunic.library.frontdesk_room_fqid_counts',
 'tunic.library.microfiche.reader.paper2.bingo_ET_mean_',
 'tunic.library.microfiche_ET_max_',
 'tunic.library_ET_mean_',
 'wellsbadge_ET_max_',
 'wellsbadge_ET_sum_',
 'worker_ET_sum_'},

13:{'Hey, this is Youmans!_text_ET_max_',
 'Hey, this is Youmans!_text_ET_sum_',
 "It's a match!_text_ET_max_",
 "It's a match!_text_ET_sum_",
 'Youmans was a suffragist!_text_ET_sum_',
 'basic_name_counts',
 'hover_duration_max_',
 'journals.hub.topics_ET_max_',
 'journals.hub.topics_ET_sum_',
 'journals.pic_1.next_ET_mean_',
 'journals.pic_1.next_ET_sum_',
 'journals.pic_2.next_ET_sum_',
 'logbook.page.bingo_ET_sum_',
 'notification_click_ET_mean_',
 'object_click_event_name_counts',
 'object_hover_ET_max_',
 'object_hover_ET_mean_',
 'reader.paper0.next_ET_sum_',
 'reader.paper0.prev_ET_mean_',
 'reader.paper1.next_ET_max_',
 'reader.paper1.next_ET_mean_',
 'reader.paper1.next_ET_sum_',
 'room_fqid_unique_',
 'tobasement_ET_max_',
 'tunic.capitol_0.hall_room_fqid_counts',
 'tunic.capitol_1.hall_room_fqid_counts',
 'tunic.drycleaner.frontdesk.logbook.page.bingo_ET_mean_',
 'tunic.historicalsociety.closet_dirty.what_happened_ET_mean_',
 'tunic.historicalsociety.frontdesk.archivist.have_glass_recap_ET_mean_',
 'tunic.historicalsociety.stacks.journals.pic_2.bingo_ET_mean_',
 'tunic.historicalsociety.stacks.journals.pic_2.bingo_ET_sum_',
 'tunic.humanecology.frontdesk.businesscards.card_bingo.bingo_ET_mean_',
 'tunic.library.microfiche.reader.paper2.bingo_ET_sum_'},
3:{'A boring old shirt._text_ET_max_',
 'Besides, I already figured out the shirt._text_ET_max_',
 'Besides, I already figured out the shirt._text_ET_sum_',
 'Better check back later._text_ET_sum_',
 'Can I come, Gramps?_text_ET_max_',
 'Can I come, Gramps?_text_ET_sum_',
 'Can I take a closer look?_text_ET_max_',
 'Could be. But we need evidence!_text_ET_sum_',
 'Did you do all of them?_text_ET_max_',
 'Did you do all of them?_text_ET_sum_',
 'Find anything?_text_ET_max_',
 'Find anything?_text_ET_sum_',
 'Found it!_text_ET_max_',
 'Found it!_text_ET_sum_',
 'Go ahead, take a peek at the shirt!_text_ET_max_',
 'Go ahead, take a peek at the shirt!_text_ET_sum_',
 'Grab your notebook and come upstairs!_text_ET_max_',
 'Gramps is a great historian!_text_ET_max_',
 'Gramps is in trouble for losing papers?_text_ET_max_',
 'Gramps is in trouble for losing papers?_text_ET_sum_',
 'Gramps is the best historian ever!_text_ET_max_',
 'Head over to the Basketball Center._text_ET_max_',
 'Head over to the Basketball Center._text_ET_sum_',
 "Hey Jo, let's take a look at the shirt!_text_ET_max_",
 "Hmm. Button's still not working._text_ET_sum_",
 "Hmmm. Don't forget about your homework._text_ET_max_",
 'Hooray, a boring old shirt._text_ET_max_',
 'Hooray, a boring old shirt._text_ET_sum_',
 'Hopefully you can rustle up some clues!_text_ET_max_',
 'Hot Dog! I knew it!_text_ET_max_',
 'Hot Dog! I knew it!_text_ET_sum_',
 "I get to go to Gramps's meeting!_text_ET_max_",
 "I get to go to Gramps's meeting!_text_ET_sum_",
 'I gotta run to my meeting!_text_ET_max_',
 'I gotta run to my meeting!_text_ET_sum_',
 'I knew it!_text_ET_max_',
 'I need to get to the Capitol and tell Gramps!_text_ET_max_',
 'I need to get to the Capitol and tell Gramps!_text_ET_sum_',
 'I suppose historians are boring, too?_text_ET_max_',
 'I suppose historians are boring, too?_text_ET_sum_',
 "I'll be at the Capitol. Let me know if you find anything!_text_ET_max_",
 "I'll be at the Capitol. Let me know if you find anything!_text_ET_sum_",
 "I'll hurry back and then we can go exploring!_text_ET_max_",
 "I'm not so sure that this is a basketball jersey._text_ET_sum_",
 "It's a women's basketball jersey!_text_ET_max_",
 "It's already all done!_text_ET_max_",
 "It's true, they do keep going missing lately._text_ET_max_",
 "It's true, they do keep going missing lately._text_ET_sum_",
 'Just talking to Teddy._text_ET_max_',
 'Just talking to Teddy._text_ET_sum_',
 'Just this old slip from 1916._text_ET_max_',
 "Leopold, why don't you help me set up in the Capitol?_text_ET_max_",
 "Let's get started. The Wisconsin Wonders exhibit opens tomorrow!_text_ET_max_",
 "Let's get started. The Wisconsin Wonders exhibit opens tomorrow!_text_ET_sum_",
 "Look at that! It's the bee's knees!_text_ET_max_",
 'None_text_ET_max_',
 'None_text_ET_sum_',
 'None_text_counts',
 'Now where did I put my notebook?_text_ET_max_',
 'Now where did I put my notebook?_text_ET_sum_',
 'Ooh, I like clues!_text_ET_max_',
 'Ooh, I like clues!_text_ET_sum_',
 'Our shirt is too old to be a basketball jersey!_text_ET_max_',
 'Our shirt is too old to be a basketball jersey!_text_ET_sum_',
 'Plus, my teacher said I could help you out for extra credit!_text_ET_sum_',
 'See you later, Teddy._text_ET_max_',
 'See?_text_ET_sum_',
 'Sure!_text_ET_max_',
 'Teddy and I were gonna go climb that huge tree out back!_text_ET_max_',
 'That settles it._text_ET_max_',
 "That's it!_text_ET_sum_",
 "The slip is from 1916 but the team didn't start until 1974!_text_ET_max_",
 "The slip is from 1916 but the team didn't start until 1974!_text_ET_sum_",
 'Then do it for me!_text_ET_max_',
 'Then do it for me!_text_ET_sum_',
 'This button never works!_text_ET_sum_',
 "This can't be right!_text_ET_max_",
 "This can't be right!_text_ET_sum_",
 'This looks like a clue!_text_ET_max_',
 'This looks like a clue!_text_ET_sum_',
 'Um... what did you want me to do again?_text_ET_max_',
 'Um... what did you want me to do again?_text_ET_sum_',
 'Wait, you mean Wells is wrong?!_text_ET_max_',
 'Wait, you mean Wells is wrong?!_text_ET_sum_',
 'We need to talk about that missing paperwork._text_ET_max_',
 'We need to talk about that missing paperwork._text_ET_sum_',
 "Well, I did SOME of those. I just couldn't find them!_text_ET_sum_",
 "Well, that's good enough for me._text_ET_max_",
 'Wells, finish up your report._text_ET_max_',
 'Wells, finish up your report._text_ET_sum_',
 'What a fascinating artifact!_text_ET_max_',
 'What a fascinating artifact!_text_ET_sum_',
 'Whatcha doing over there, Jo?_text_ET_max_',
 'Whatcha doing over there, Jo?_text_ET_sum_',
 'Who wants to investigate the shirt artifact?_text_ET_max_',
 'Who wants to investigate the shirt artifact?_text_ET_sum_',
 "Why don't you head to the Basketball Center and rustle up some clues?_text_ET_max_",
 "Why don't you head to the Basketball Center and rustle up some clues?_text_ET_sum_",
 'Will do, Boss._text_ET_max_',
 'Will do, Boss._text_ET_sum_',
 "Wow, that's so cool, Gramps!_text_ET_max_",
 "Wow, that's so cool, Gramps!_text_ET_sum_",
 'Yes! This old slip from 1916._text_ET_sum_',
 'Your grampa is waiting for you in the collection room._text_ET_max_',
 'Your teacher said you could help me for extra credit._text_ET_max_',
 'Your teacher said you could help me for extra credit._text_ET_sum_',
 'Your teacher said you missed 7 assignments in a row!_text_ET_max_',
 '\\u00f0\\u0178\\u02dc\\u00b4_text_ET_sum_',
 'basic_ET_max_',
 'basic_ET_mean_',
 'basic_name_counts',
 'chap1_finale_ET_mean_',
 'chap1_finale_ET_sum_',
 'chap1_finale_c_ET_max_',
 'chap1_finale_c_ET_mean_',
 'chap1_finale_c_ET_sum_',
 'chap1_finale_fqid_counts',
 'checkpoint_ET_mean_',
 'close_ET_mean_',
 'cs_ET_max_',
 'cs_ET_sum_',
 'cs_fqid_counts',
 'cutscene_click_ET_max_',
 'cutscene_click_ET_max_x',
 'cutscene_click_ET_mean_x',
 'cutscene_click_event_name_counts',
 'directory_ET_max_',
 'directory_ET_mean_',
 'directory_ET_sum_',
 'directory_fqid_counts',
 'doorblock_ET_max_',
 'doorblock_ET_sum_',
 'doorblock_fqid_counts',
 'elapsed_time_diff_max_',
 'elapsed_time_diff_mean_',
 'gramps_ET_max_',
 'gramps_ET_mean_',
 'groupconvo_ET_mean_',
 'groupconvo_ET_sum_',
 'hover_duration_max_',
 'hover_duration_mean_',
 'intro_ET_max_',
 'intro_ET_mean_',
 'intro_ET_sum_',
 'janitor_ET_max_',
 'map_click_ET_max_',
 'map_click_ET_max_x',
 'map_click_ET_mean_',
 'map_click_ET_mean_x',
 'map_hover_ET_max_',
 'map_hover_ET_mean_',
 'navigate_click_ET_max_',
 'navigate_click_ET_mean_x',
 'navigate_click_event_name_counts',
 'notebook_ET_max_',
 'notebook_ET_mean_',
 'notebook_ET_sum_',
 'notebook_click_ET_max_',
 'notebook_click_ET_max_x',
 'notebook_click_ET_mean_',
 'notebook_click_ET_mean_x',
 'notification_click_ET_max_',
 'notification_click_ET_max_x',
 'notification_click_ET_mean_x',
 'object_click_ET_max_',
 'object_click_ET_max_x',
 'object_click_ET_mean_',
 'object_click_ET_mean_x',
 'object_hover_ET_max_',
 'object_hover_ET_mean_',
 'object_hover_event_name_counts',
 'observation_click_ET_max_',
 'observation_click_ET_max_x',
 'observation_click_ET_mean_',
 'observation_click_ET_mean_x',
 'observation_click_event_name_counts',
 'open_ET_max_',
 'open_ET_mean_',
 'open_name_counts',
 'outtolunch_ET_sum_',
 'outtolunch_fqid_counts',
 'page_max_',
 'page_mean_',
 'person_click_ET_max_x',
 'person_click_ET_mean_',
 'person_click_ET_mean_x',
 'person_click_event_name_counts',
 'photo_ET_max_',
 'photo_ET_sum_',
 'plaque.face.date_ET_max_',
 'plaque.face.date_ET_mean_',
 'plaque.face.date_ET_sum_',
 'plaque.face.date_fqid_counts',
 'plaque_ET_mean_',
 'plaque_fqid_counts',
 'report_ET_max_',
 'report_ET_mean_',
 'report_ET_sum_',
 'report_fqid_counts',
 'retirement_letter_ET_max_',
 'retirement_letter_ET_mean_',
 'retirement_letter_fqid_counts',
 'room_coor_x_max_',
 'room_coor_x_mean_',
 'screen_coor_x_mean_',
 'screen_coor_y_max_',
 'screen_coor_y_mean_',
 'session_number_',
 'teddy_ET_max_',
 'teddy_ET_mean_',
 'teddy_ET_sum_',
 'tobasement_ET_max_',
 'tobasement_ET_mean_',
 'tobasement_ET_sum_',
 'tobasement_fqid_counts',
 'tocloset_ET_mean_',
 'tocollection_ET_mean_',
 'tocollection_ET_sum_',
 'toentry_ET_mean_',
 'toentry_ET_sum_',
 'toentry_fqid_counts',
 'togrampa_ET_max_',
 'tomap_ET_mean_',
 'tomap_ET_sum_',
 'tomap_fqid_counts',
 'tostacks_ET_max_',
 'tostacks_ET_mean_',
 'tostacks_ET_sum_',
 'tunic.capitol_0.hall.chap1_finale_c_text_fqid_counts',
 'tunic.capitol_0.hall_ET_max_',
 'tunic.capitol_0.hall_ET_mean_',
 'tunic.capitol_0.hall_room_fqid_counts',
 'tunic.capitol_0_ET_mean_',
 'tunic.capitol_0_ET_sum_',
 'tunic.historicalsociety.basement.janitor_ET_max_',
 'tunic.historicalsociety.basement.janitor_ET_mean_',
 'tunic.historicalsociety.basement_ET_max_',
 'tunic.historicalsociety.basement_ET_mean_',
 'tunic.historicalsociety.basement_ET_sum_',
 'tunic.historicalsociety.basement_room_fqid_counts',
 'tunic.historicalsociety.closet.doorblock_ET_max_',
 'tunic.historicalsociety.closet.doorblock_ET_mean_',
 'tunic.historicalsociety.closet.gramps.intro_0_cs_0_ET_max_',
 'tunic.historicalsociety.closet.gramps.intro_0_cs_0_ET_mean_',
 'tunic.historicalsociety.closet.gramps.intro_0_cs_0_ET_sum_',
 'tunic.historicalsociety.closet.intro_ET_sum_',
 'tunic.historicalsociety.closet.notebook_ET_mean_',
 'tunic.historicalsociety.closet.retirement_letter.hub_ET_mean_',
 'tunic.historicalsociety.closet.teddy.intro_0_cs_0_ET_max_',
 'tunic.historicalsociety.closet.teddy.intro_0_cs_0_ET_sum_',
 'tunic.historicalsociety.closet.teddy.intro_0_cs_5_ET_max_',
 'tunic.historicalsociety.closet.teddy.intro_0_cs_5_ET_mean_',
 'tunic.historicalsociety.closet_ET_max_',
 'tunic.historicalsociety.closet_ET_mean_',
 'tunic.historicalsociety.closet_ET_sum_',
 'tunic.historicalsociety.closet_room_fqid_counts',
 'tunic.historicalsociety.collection.cs_ET_max_',
 'tunic.historicalsociety.collection.gramps.lost_ET_max_',
 'tunic.historicalsociety.collection.tunic.slip_ET_max_',
 'tunic.historicalsociety.collection.tunic.slip_ET_mean_',
 'tunic.historicalsociety.collection.tunic_ET_mean_',
 'tunic.historicalsociety.collection.tunic_ET_sum_',
 'tunic.historicalsociety.collection_ET_max_',
 'tunic.historicalsociety.collection_room_fqid_counts',
 'tunic.historicalsociety.entry.boss.talktogramps_ET_mean_',
 'tunic.historicalsociety.entry.groupconvo_ET_max_',
 'tunic.historicalsociety.entry.groupconvo_ET_sum_',
 'tunic.historicalsociety.entry_ET_max_',
 'tunic.historicalsociety.entry_room_fqid_counts',
 'tunic.historicalsociety.stacks.outtolunch_ET_mean_',
 'tunic.historicalsociety.stacks_ET_sum_',
 'tunic.historicalsociety_ET_max_',
 'tunic.historicalsociety_ET_mean_',
 'tunic.hub.slip_ET_max_',
 'tunic.hub.slip_ET_mean_',
 'tunic.hub.slip_ET_sum_',
 'tunic.kohlcenter.halloffame.plaque.face.date_ET_max_',
 'tunic.kohlcenter.halloffame.plaque.face.date_ET_sum_',
 'tunic.kohlcenter.halloffame.togrampa_ET_max_',
 'tunic.kohlcenter.halloffame.togrampa_ET_mean_',
 'tunic.kohlcenter.halloffame_ET_max_',
 'tunic.kohlcenter.halloffame_ET_mean_',
 'tunic.kohlcenter.halloffame_ET_sum_',
 'tunic.kohlcenter.halloffame_room_fqid_counts',
 'tunic.kohlcenter_ET_max_',
 'tunic.kohlcenter_ET_mean_',
 'tunic.kohlcenter_ET_sum_',
 'tunic_ET_max_',
 'tunic_ET_mean_',
 'tunic_ET_sum_',
 'tunic_fqid_counts',
 'undefined_ET_max_',
 'undefined_text_ET_sum_',
 'wells_ET_mean_',
 'wells_ET_sum_',
 'wells_fqid_counts'}}
for q in i_features:
    print(q)
    features_use_because_of_importances[q]=list( i_features[q])

12
13
3


In [5]:
TRAIN=True

In [6]:

dtypes={"session_id": pl.Int64,
          "elapsed_time": pl.Int64,
          "event_name": pl.Categorical,
          "name": pl.Categorical,
          "page": pl.Float32,
          "room_coor_x": pl.Float32,
          "room_coor_y": pl.Float32,
          "screen_coor_x": pl.Float32,
          "screen_coor_y": pl.Float32,
          "hover_duration": pl.Float32,
          "text": pl.Categorical,
          "fqid": pl.Categorical,
          "room_fqid": pl.Categorical,
          "text_fqid": pl.Categorical,
          "fullscreen":  pl.Float32,
          "hq": pl.Float32,
          "music": pl.Float32,
          "level_group": pl.Categorical
          }


# Data preprocessing

In [7]:
CATS = ['event_name', 'name', 'fqid', 'room_fqid', 'text_fqid']
NUMS = ['page', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y',
        'hover_duration', 'elapsed_time_diff']

name_feature = ['basic', 'undefined', 'close', 'open', 'prev', 'next']
event_name_feature = ['cutscene_click', 'person_click', 'navigate_click',
       'observation_click', 'notification_click', 'object_click',
       'object_hover', 'map_hover', 'map_click', 'checkpoint',
       'notebook_click']

# from https://www.kaggle.com/code/leehomhuang/catboost-baseline-with-lots-features-inference :
fqid_lists = ['worker', 'archivist', 'gramps', 'wells', 'toentry', 'confrontation', 'crane_ranger', 'groupconvo', 'flag_girl', 'tomap', 'tostacks', 'tobasement', 'archivist_glasses', 'boss', 'journals', 'seescratches', 'groupconvo_flag', 'cs', 'teddy', 'expert', 'businesscards', 'ch3start', 'tunic.historicalsociety', 'tofrontdesk', 'savedteddy', 'plaque', 'glasses', 'tunic.drycleaner', 'reader_flag', 'tunic.library', 'tracks', 'tunic.capitol_2', 'trigger_scarf', 'reader', 'directory', 'tunic.capitol_1', 'journals.pic_0.next', 'unlockdoor', 'tunic', 'what_happened', 'tunic.kohlcenter', 'tunic.humanecology', 'colorbook', 'logbook', 'businesscards.card_0.next', 'journals.hub.topics', 'logbook.page.bingo', 'journals.pic_1.next', 'journals_flag', 'reader.paper0.next', 'tracks.hub.deer', 'reader_flag.paper0.next', 'trigger_coffee', 'wellsbadge', 'journals.pic_2.next', 'tomicrofiche', 'journals_flag.pic_0.bingo', 'plaque.face.date', 'notebook', 'tocloset_dirty', 'businesscards.card_bingo.bingo', 'businesscards.card_1.next', 'tunic.wildlife', 'tunic.hub.slip', 'tocage', 'journals.pic_2.bingo', 'tocollectionflag', 'tocollection', 'chap4_finale_c', 'chap2_finale_c', 'lockeddoor', 'journals_flag.hub.topics', 'tunic.capitol_0', 'reader_flag.paper2.bingo', 'photo', 'tunic.flaghouse', 'reader.paper1.next', 'directory.closeup.archivist', 'intro', 'businesscards.card_bingo.next', 'reader.paper2.bingo', 'retirement_letter', 'remove_cup', 'journals_flag.pic_0.next', 'magnify', 'coffee', 'key', 'togrampa', 'reader_flag.paper1.next', 'janitor', 'tohallway', 'chap1_finale', 'report', 'outtolunch', 'journals_flag.hub.topics_old', 'journals_flag.pic_1.next', 'reader.paper2.next', 'chap1_finale_c', 'reader_flag.paper2.next', 'door_block_talk', 'journals_flag.pic_1.bingo', 'journals_flag.pic_2.next', 'journals_flag.pic_2.bingo', 'block_magnify', 'reader.paper0.prev', 'block', 'reader_flag.paper0.prev', 'block_0', 'door_block_clean', 'reader.paper2.prev', 'reader.paper1.prev', 'doorblock', 'tocloset', 'reader_flag.paper2.prev', 'reader_flag.paper1.prev', 'block_tomap2', 'journals_flag.pic_0_old.next', 'journals_flag.pic_1_old.next', 'block_tocollection', 'block_nelson', 'journals_flag.pic_2_old.next', 'block_tomap1', 'block_badge', 'need_glasses', 'block_badge_2', 'fox', 'block_1']
text_lists = ['tunic.historicalsociety.cage.confrontation', 'tunic.wildlife.center.crane_ranger.crane', 'tunic.historicalsociety.frontdesk.archivist.newspaper', 'tunic.historicalsociety.entry.groupconvo', 'tunic.wildlife.center.wells.nodeer', 'tunic.historicalsociety.frontdesk.archivist.have_glass', 'tunic.drycleaner.frontdesk.worker.hub', 'tunic.historicalsociety.closet_dirty.gramps.news', 'tunic.humanecology.frontdesk.worker.intro', 'tunic.historicalsociety.frontdesk.archivist_glasses.confrontation', 'tunic.historicalsociety.basement.seescratches', 'tunic.historicalsociety.collection.cs', 'tunic.flaghouse.entry.flag_girl.hello', 'tunic.historicalsociety.collection.gramps.found', 'tunic.historicalsociety.basement.ch3start', 'tunic.historicalsociety.entry.groupconvo_flag', 'tunic.library.frontdesk.worker.hello', 'tunic.library.frontdesk.worker.wells', 'tunic.historicalsociety.collection_flag.gramps.flag', 'tunic.historicalsociety.basement.savedteddy', 'tunic.library.frontdesk.worker.nelson', 'tunic.wildlife.center.expert.removed_cup', 'tunic.library.frontdesk.worker.flag', 'tunic.historicalsociety.frontdesk.archivist.hello', 'tunic.historicalsociety.closet.gramps.intro_0_cs_0', 'tunic.historicalsociety.entry.boss.flag', 'tunic.flaghouse.entry.flag_girl.symbol', 'tunic.historicalsociety.closet_dirty.trigger_scarf', 'tunic.drycleaner.frontdesk.worker.done', 'tunic.historicalsociety.closet_dirty.what_happened', 'tunic.wildlife.center.wells.animals', 'tunic.historicalsociety.closet.teddy.intro_0_cs_0', 'tunic.historicalsociety.cage.glasses.afterteddy', 'tunic.historicalsociety.cage.teddy.trapped', 'tunic.historicalsociety.cage.unlockdoor', 'tunic.historicalsociety.stacks.journals.pic_2.bingo', 'tunic.historicalsociety.entry.wells.flag', 'tunic.humanecology.frontdesk.worker.badger', 'tunic.historicalsociety.stacks.journals_flag.pic_0.bingo', 'tunic.historicalsociety.closet.intro', 'tunic.historicalsociety.closet.retirement_letter.hub', 'tunic.historicalsociety.entry.directory.closeup.archivist', 'tunic.historicalsociety.collection.tunic.slip', 'tunic.kohlcenter.halloffame.plaque.face.date', 'tunic.historicalsociety.closet_dirty.trigger_coffee', 'tunic.drycleaner.frontdesk.logbook.page.bingo', 'tunic.library.microfiche.reader.paper2.bingo', 'tunic.kohlcenter.halloffame.togrampa', 'tunic.capitol_2.hall.boss.haveyougotit', 'tunic.wildlife.center.wells.nodeer_recap', 'tunic.historicalsociety.cage.glasses.beforeteddy', 'tunic.historicalsociety.closet_dirty.gramps.helpclean', 'tunic.wildlife.center.expert.recap', 'tunic.historicalsociety.frontdesk.archivist.have_glass_recap', 'tunic.historicalsociety.stacks.journals_flag.pic_1.bingo', 'tunic.historicalsociety.cage.lockeddoor', 'tunic.historicalsociety.stacks.journals_flag.pic_2.bingo', 'tunic.historicalsociety.collection.gramps.lost', 'tunic.historicalsociety.closet.notebook', 'tunic.historicalsociety.frontdesk.magnify', 'tunic.humanecology.frontdesk.businesscards.card_bingo.bingo', 'tunic.wildlife.center.remove_cup', 'tunic.library.frontdesk.wellsbadge.hub', 'tunic.wildlife.center.tracks.hub.deer', 'tunic.historicalsociety.frontdesk.key', 'tunic.library.microfiche.reader_flag.paper2.bingo', 'tunic.flaghouse.entry.colorbook', 'tunic.wildlife.center.coffee', 'tunic.capitol_1.hall.boss.haveyougotit', 'tunic.historicalsociety.basement.janitor', 'tunic.historicalsociety.collection_flag.gramps.recap', 'tunic.wildlife.center.wells.animals2', 'tunic.flaghouse.entry.flag_girl.symbol_recap', 'tunic.historicalsociety.closet_dirty.photo', 'tunic.historicalsociety.stacks.outtolunch', 'tunic.library.frontdesk.worker.wells_recap', 'tunic.historicalsociety.frontdesk.archivist_glasses.confrontation_recap', 'tunic.capitol_0.hall.boss.talktogramps', 'tunic.historicalsociety.closet.photo', 'tunic.historicalsociety.collection.tunic', 'tunic.historicalsociety.closet.teddy.intro_0_cs_5', 'tunic.historicalsociety.closet_dirty.gramps.archivist', 'tunic.historicalsociety.closet_dirty.door_block_talk', 'tunic.historicalsociety.entry.boss.flag_recap', 'tunic.historicalsociety.frontdesk.archivist.need_glass_0', 'tunic.historicalsociety.entry.wells.talktogramps', 'tunic.historicalsociety.frontdesk.block_magnify', 'tunic.historicalsociety.frontdesk.archivist.foundtheodora', 'tunic.historicalsociety.closet_dirty.gramps.nothing', 'tunic.historicalsociety.closet_dirty.door_block_clean', 'tunic.capitol_1.hall.boss.writeitup', 'tunic.library.frontdesk.worker.nelson_recap', 'tunic.library.frontdesk.worker.hello_short', 'tunic.historicalsociety.stacks.block', 'tunic.historicalsociety.frontdesk.archivist.need_glass_1', 'tunic.historicalsociety.entry.boss.talktogramps', 'tunic.historicalsociety.frontdesk.archivist.newspaper_recap', 'tunic.historicalsociety.entry.wells.flag_recap', 'tunic.drycleaner.frontdesk.worker.done2', 'tunic.library.frontdesk.worker.flag_recap', 'tunic.humanecology.frontdesk.block_0', 'tunic.library.frontdesk.worker.preflag', 'tunic.historicalsociety.basement.gramps.seeyalater', 'tunic.flaghouse.entry.flag_girl.hello_recap', 'tunic.historicalsociety.closet.doorblock', 'tunic.drycleaner.frontdesk.worker.takealook', 'tunic.historicalsociety.basement.gramps.whatdo', 'tunic.library.frontdesk.worker.droppedbadge', 'tunic.historicalsociety.entry.block_tomap2', 'tunic.library.frontdesk.block_nelson', 'tunic.library.microfiche.block_0', 'tunic.historicalsociety.entry.block_tocollection', 'tunic.historicalsociety.entry.block_tomap1', 'tunic.historicalsociety.collection.gramps.look_0', 'tunic.library.frontdesk.block_badge', 'tunic.historicalsociety.cage.need_glasses', 'tunic.library.frontdesk.block_badge_2', 'tunic.kohlcenter.halloffame.block_0', 'tunic.capitol_0.hall.chap1_finale_c', 'tunic.capitol_1.hall.chap2_finale_c', 'tunic.capitol_2.hall.chap4_finale_c', 'tunic.wildlife.center.fox.concern', 'tunic.drycleaner.frontdesk.block_0', 'tunic.historicalsociety.entry.gramps.hub', 'tunic.humanecology.frontdesk.block_1', 'tunic.drycleaner.frontdesk.block_1']
room_lists = ['tunic.historicalsociety.entry', 'tunic.wildlife.center', 'tunic.historicalsociety.cage', 'tunic.library.frontdesk', 'tunic.historicalsociety.frontdesk', 'tunic.historicalsociety.stacks', 'tunic.historicalsociety.closet_dirty', 'tunic.humanecology.frontdesk', 'tunic.historicalsociety.basement', 'tunic.kohlcenter.halloffame', 'tunic.library.microfiche', 'tunic.drycleaner.frontdesk', 'tunic.historicalsociety.collection', 'tunic.historicalsociety.closet', 'tunic.flaghouse.entry', 'tunic.historicalsociety.collection_flag', 'tunic.capitol_1.hall', 'tunic.capitol_0.hall', 'tunic.capitol_2.hall']

In [8]:
for i in features_use_because_of_importances:
    print(i,len(features_use_because_of_importances[i]))

1 356
4 492
5 532
6 602
7 447
8 373
9 597
10 684
11 525
14 321
15 630
16 1690
17 405
2 8
3 296
12 195
13 33
18 1895


In [9]:
columns = [

    pl.col("page").cast(pl.Float32),
    (
        (pl.col("elapsed_time") - pl.col("elapsed_time").shift(1)) 
         .fill_null(0)
         .clip(0, 1e9)
         .over(["session_id", "level_group"])
         .alias("elapsed_time_diff")
    ),
    (
        (pl.col("screen_coor_x") - pl.col("screen_coor_x").shift(1)) 
         .abs()
         .over(["session_id", "level_group"])
        .alias("location_x_diff") 
    ),
    (
        (pl.col("screen_coor_y") - pl.col("screen_coor_y").shift(1)) 
         .abs()
         .over(["session_id", "level_group"])
        .alias("location_y_diff") 
    ),
    
]

In [10]:
# we prepare the dataset for the training by level :
df = (pl.read_csv("/kaggle/input/predict-student-performance-from-game-play/train.csv",dtypes=dtypes)
      .drop(["fullscreen", "hq", "music"])
      .with_columns(columns))


texts=list(df["text"].unique())
len(texts)

598

In [11]:
def feature_engineer_pl(x, grp, use_extra, feature_suffix):
        
    aggs = [
        pl.col("index").count().alias(f"session_number_{feature_suffix}"),
      
        *[pl.col(c).drop_nulls().n_unique().alias(f"{c}_unique_{feature_suffix}") for c in CATS],
        *[pl.col(c).quantile(0.1, "nearest").alias(f"{c}_quantile1_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.2, "nearest").alias(f"{c}_quantile2_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.4, "nearest").alias(f"{c}_quantile4_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.6, "nearest").alias(f"{c}_quantile6_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.8, "nearest").alias(f"{c}_quantile8_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.9, "nearest").alias(f"{c}_quantile9_{feature_suffix}") for c in NUMS],
        
        *[pl.col(c).mean().alias(f"{c}_mean_{feature_suffix}") for c in NUMS],
        *[pl.col(c).std().alias(f"{c}_std_{feature_suffix}") for c in NUMS],
        *[pl.col(c).min().alias(f"{c}_min_{feature_suffix}") for c in NUMS],
        *[pl.col(c).max().alias(f"{c}_max_{feature_suffix}") for c in NUMS],
        *[pl.col(c).sum().alias(f"{c}_sum_{feature_suffix}") for c in NUMS],
        *[pl.col(c).apply(lambda x: feature_calculators.kurtosis(x)).alias(f"{c}_kurtosis_{feature_suffix}") for c in NUMS],
        *[pl.col(c).apply(lambda x: feature_calculators.skewness(x)).alias(f"{c}_skewness_{feature_suffix}") for c in NUMS],
        *[pl.col(c).apply(lambda x: feature_calculators.first_location_of_maximum(x)).alias(f"{c}_first_position_maximun_{feature_suffix}") for c in NUMS],
        *[pl.col(c).apply(lambda x: feature_calculators.last_location_of_maximum(x)).alias(f"{c}_last_position_maximun_{feature_suffix}") for c in NUMS],
        
        *[pl.col("event_name").filter(pl.col("event_name") == c).count().alias(f"{c}_event_name_counts{feature_suffix}")for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).quantile(0.1, "nearest").alias(f"{c}_ET_quantile1_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).quantile(0.2, "nearest").alias(f"{c}_ET_quantile2_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).quantile(0.4, "nearest").alias(f"{c}_ET_quantile4_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).quantile(0.6, "nearest").alias(f"{c}_ET_quantile6_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).quantile(0.8, "nearest").alias(f"{c}_ET_quantile8_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).quantile(0.9, "nearest").alias(f"{c}_ET_quantile9_{feature_suffix}") for c in event_name_feature],      
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).min().alias(f"{c}_ET_min_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name")==c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name") == c).apply(lambda x: feature_calculators.kurtosis(x)).alias(f" {c}_ET_kurtosis_{feature_suffix}") for c in event_name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("event_name") == c).apply(lambda x: feature_calculators.skewness(x)).alias(f"{c}_ET_skewness_{feature_suffix}") for c in event_name_feature],
     
        *[pl.col("name").filter(pl.col("name") == c).count().alias(f"{c}_name_counts{feature_suffix}")for c in name_feature],   
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).min().alias(f"{c}_ET_min_{feature_suffix}") for c in name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in name_feature],  
        *[pl.col("elapsed_time_diff").filter(pl.col("name")==c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in name_feature],  
        *[pl.col("elapsed_time_diff").filter(pl.col("name") == c).apply(lambda x: feature_calculators.kurtosis(x)).alias(f" {c}_ET_kurtosis_{feature_suffix}") for c in name_feature],
        *[pl.col("elapsed_time_diff").filter(pl.col("name") == c).apply(lambda x: feature_calculators.skewness(x)).alias(f"{c}_ET_skewness_{feature_suffix}") for c in name_feature],
        
        *[pl.col("room_fqid").filter(pl.col("room_fqid") == c).count().alias(f"{c}_room_fqid_counts{feature_suffix}")for c in room_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in room_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in room_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in room_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).min().alias(f"{c}_ET_min_{feature_suffix}") for c in room_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("room_fqid") == c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in room_lists],
                
        *[pl.col("fqid").filter(pl.col("fqid") == c).count().alias(f"{c}_fqid_counts{feature_suffix}")for c in fqid_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in fqid_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in fqid_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in fqid_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).min().alias(f"{c}_ET_min_{feature_suffix}") for c in fqid_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("fqid") == c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in fqid_lists],
       
        *[pl.col("text_fqid").filter(pl.col("text_fqid") == c).count().alias(f"{c}_text_fqid_counts{feature_suffix}") for c in text_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).std().alias(f"{c}_ET_std_{feature_suffix}") for c in text_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).mean().alias(f"{c}_ET_mean_{feature_suffix}") for c in text_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).max().alias(f"{c}_ET_max_{feature_suffix}") for c in text_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).min().alias(f"{c}_ET_min_{feature_suffix}") for c in text_lists],
        *[pl.col("elapsed_time_diff").filter(pl.col("text_fqid") == c).sum().alias(f"{c}_ET_sum_{feature_suffix}") for c in text_lists],
         
        *[pl.col("location_x_diff").filter(pl.col("event_name")==c).mean().alias(f"{c}_ET_mean_x{feature_suffix}") for c in event_name_feature],
        *[pl.col("location_x_diff").filter(pl.col("event_name")==c).std().alias(f"{c}_ET_std_x{feature_suffix}") for c in event_name_feature],
        *[pl.col("location_x_diff").filter(pl.col("event_name")==c).max().alias(f"{c}_ET_max_x{feature_suffix}") for c in event_name_feature],
        *[pl.col("location_x_diff").filter(pl.col("event_name")==c).min().alias(f"{c}_ET_min_x{feature_suffix}") for c in event_name_feature],
        *[pl.col("text_fqid").filter(pl.col("text") == c).count().alias(f"{c}_text_counts{feature_suffix}") for c in texts],
        *[pl.col("elapsed_time_diff").filter(pl.col("text") == c).std().alias(f"{c}_text_ET_std_{feature_suffix}") for c in texts],
        *[pl.col("elapsed_time_diff").filter(pl.col("text") == c).mean().alias(f"{c}_text_ET_mean_{feature_suffix}") for c in texts],
        *[pl.col("elapsed_time_diff").filter(pl.col("text") == c).max().alias(f"{c}_text_ET_max_{feature_suffix}") for c in texts],
        *[pl.col("elapsed_time_diff").filter(pl.col("text") == c).min().alias(f"{c}_text_ET_min_{feature_suffix}") for c in texts],
        *[pl.col("elapsed_time_diff").filter(pl.col("text") == c).sum().alias(f"{c}_text_ET_sum_{feature_suffix}") for c in texts],
        *[pl.col("text").filter(pl.col("text").is_null()).count().alias(f"null_text_counts{feature_suffix}")],
        *[pl.col("elapsed_time_diff").filter(pl.col("text").is_null()).std().alias(f"null_text_ET_std_{feature_suffix}")],
        *[pl.col("elapsed_time_diff").filter(pl.col("text").is_null()).mean().alias(f"null_text_ET_mean_{feature_suffix}")],
        *[pl.col("elapsed_time_diff").filter(pl.col("text").is_null()).max().alias(f"null_text_ET_max_{feature_suffix}")],
        *[pl.col("elapsed_time_diff").filter(pl.col("text").is_null()).min().alias(f"null_text_ET_min_{feature_suffix}")],
        *[pl.col("elapsed_time_diff").filter(pl.col("text").is_null()).sum().alias(f"null_text_ET_sum_{feature_suffix}")],
        
        ]
    
    df = x.groupby(["session_id"], maintain_order=True).agg(aggs).sort("session_id")
  
    if use_extra:
        if grp=='5-12':
            aggs = [
                pl.col("elapsed_time").filter((pl.col("text")=="Here's the log book.")|(pl.col("fqid")=='logbook.page.bingo')).apply(lambda s: s.max()-s.min()).alias("logbook_bingo_duration"),
                pl.col("index").filter((pl.col("text")=="Here's the log book.")|(pl.col("fqid")=='logbook.page.bingo')).apply(lambda s: s.max()-s.min()).alias("logbook_bingo_indexCount"),
                pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='reader'))|(pl.col("fqid")=="reader.paper2.bingo")).apply(lambda s: s.max()-s.min()).alias("reader_bingo_duration"),
                pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='reader'))|(pl.col("fqid")=="reader.paper2.bingo")).apply(lambda s: s.max()-s.min()).alias("reader_bingo_indexCount"),
                pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='journals'))|(pl.col("fqid")=="journals.pic_2.bingo")).apply(lambda s: s.max()-s.min()).alias("journals_bingo_duration"),
                pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='journals'))|(pl.col("fqid")=="journals.pic_2.bingo")).apply(lambda s: s.max()-s.min()).alias("journals_bingo_indexCount"),
            ]
            tmp = x.groupby(["session_id"], maintain_order=True).agg(aggs).sort("session_id")
            df = df.join(tmp, on="session_id", how='left')

        if grp=='13-22':
            aggs = [
                pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='reader_flag'))|(pl.col("fqid")=="tunic.library.microfiche.reader_flag.paper2.bingo")).apply(lambda s: s.max()-s.min() if s.len()>0 else 0).alias("reader_flag_duration"),
                pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='reader_flag'))|(pl.col("fqid")=="tunic.library.microfiche.reader_flag.paper2.bingo")).apply(lambda s: s.max()-s.min() if s.len()>0 else 0).alias("reader_flag_indexCount"),
                pl.col("elapsed_time").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='journals_flag'))|(pl.col("fqid")=="journals_flag.pic_0.bingo")).apply(lambda s: s.max()-s.min() if s.len()>0 else 0).alias("journalsFlag_bingo_duration"),
                pl.col("index").filter(((pl.col("event_name")=='navigate_click')&(pl.col("fqid")=='journals_flag'))|(pl.col("fqid")=="journals_flag.pic_0.bingo")).apply(lambda s: s.max()-s.min() if s.len()>0 else 0).alias("journalsFlag_bingo_indexCount"),
            ]
            tmp = x.groupby(["session_id"], maintain_order=True).agg(aggs).sort("session_id")
            df = df.join(tmp, on="session_id", how='left')
        
    return df.to_pandas()

In [12]:
xgb_params_by_question={1: {'params': {'best_its': (800, 800),
   'max_depth': 6,
   'scale_pos_weight': 0.39999999999999997,
   'learning_rate': 0.022,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5000000000000001},
 4: {'params': {'best_its': [(539, 691), (497, 582), (891, 867), (599, 607)],
   'max_depth': 4,
   'scale_pos_weight': 0.7,
   'learning_rate': 0.027,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5600000000000002},
 5: {'params': {'best_its': [(564, 564), (468, 478), (306, 384), (341, 478)], 'max_depth': 5, 'grow_policy': 'lossguide', 'scale_pos_weight': 0.8999999999999999, 'learning_rate': 0.022, 'colsample_bytree': 0.4, 'subsample': 0.6, 'tree_method': 'hist', 'n_jobs': 16, 'random_state': 32},
  'umbral': 0.6000000000000003},
 6: {'params': {'best_its': (650, 650),
   'max_depth': 5,
   'scale_pos_weight': 0.6,
   'learning_rate': 0.02,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5500000000000002},
 7: {'params': {'best_its': (800, 800),
   'max_depth': 4,
   'scale_pos_weight': 0.6,
   'learning_rate': 0.02,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5200000000000001},
 8: {'params': {'best_its': (350, 350),
   'max_depth': 6,
   'scale_pos_weight': 1.0999999999999999,
   'learning_rate': 0.015,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.6600000000000003},
 9: {'params': {'best_its': (699, 763),
   'max_depth': 5,
   'scale_pos_weight': 0.5,
   'learning_rate': 0.02,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.4900000000000001},
10: {'params': {'best_its': [(397, 377), (374, 381), (412, 329), (253, 336)],
   'max_depth': 6,
   'scale_pos_weight': 0.7999999999999999,
   'learning_rate': 0.02,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5700000000000002},
 11: {'params': {'best_its': (450, 450),
   'max_depth': 6,
   'scale_pos_weight': 0.7999999999999999,
   'learning_rate': 0.022,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5900000000000002},
 14: {'params': {'best_its': (450, 450),
   'max_depth': 5,
   'scale_pos_weight': 0.9999999999999999,
   'learning_rate': 0.02,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.6400000000000002},
 15: {'params': {'best_its': (700, 700),
   'max_depth': 6,
   'scale_pos_weight': 0.5,
   'learning_rate': 0.02,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5100000000000001},
 16: {'params': {'best_its': (500, 500),
   'max_depth': 4,
   'scale_pos_weight': 0.6,
   'learning_rate': 0.02,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5100000000000001},
 17: {'params': {'best_its': (195, 225),
   'max_depth': 5,
   'scale_pos_weight': 0.8999999999999999,
   'learning_rate': 0.02,
   'tree_method': 'hist',
   'n_jobs': 16,
   'random_state': 32,
   'subsample': 0.8,
   'colsample_bytree': 0.7,
   'grow_policy': 'lossguide'},
  'umbral': 0.5900000000000002},

 2: {'params': {'booster': 'gbtree', 'tree_method': 'hist', 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'learning_rate': 0.02, 'alpha': 8, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.5, 'seed': 42, 'n_estimators': 448}, 
        'umbral': 0.625, 'best_iterations': None},
 3: {'params': {'n_estimators': 296, 'max_depth': 5, 'grow_policy': 'lossguide', 'scale_pos_weight': 1.0999999999999999, 'learning_rate': 0.02, 'colsample_bytree': 0.7, 'subsample': 0.8, 'tree_method': 'hist', 'n_jobs': 16, 'random_state': 32, 'eval_metric': 'logloss', 'best_its': (292, 296)}, 
        'umbral': 0.6600000000000004, 'best_iterations': None},

 12: {'params': {'n_estimators': 236, 'max_depth': 5, 'grow_policy': 'lossguide', 'scale_pos_weight': 1.0999999999999999, 'learning_rate': 0.02, 'colsample_bytree': 0.7, 'subsample': 0.8, 'tree_method': 'hist', 'n_jobs': 16, 'random_state': 32, 'eval_metric': 'logloss', 'best_its': (197, 236)},
     'umbral': 0.6800000000000004, 'best_iterations': None},
 13: {'params': {'n_estimators': 255, 'max_depth': 5, 'grow_policy': 'lossguide', 'scale_pos_weight': 1.0999999999999999, 'learning_rate': 0.02, 'colsample_bytree': 0.7, 'subsample': 0.8, 'tree_method': 'hist', 'n_jobs': 16, 'random_state': 32, 'eval_metric': 'logloss', 'best_its': (276, 255)}, 
     'umbral': 0.6800000000000004, 'best_iterations': None}
    
    }
  

## We fit and store the models for predictions

In [13]:
QUESTIONS_BASE_FEATURES=[2,3, 9, 12,13, 17,18]
QUESTION_NEW_FEATURES= [4, 5, 8, 10, 11, 14, 15, 16]
QUESTION_NEW_FEATURES_FOLD= [1, 6, 7]
QUESTION_NEW_FEATURES_RANDOM=[8,11,14,15]
QUESTION_SAMPLE06=[14, 15]

In [14]:
#for p in xgb_params_by_question:
    #if p in QUESTIONS_BASE_FEATURES:
        #xgb_params_by_question[p]["params"]['colsample_bytree']= 0.6
        #xgb_params_by_question[p]["params"]['subsample']= 0.6

In [15]:
for p in xgb_params_by_question:
    if p in QUESTION_SAMPLE06:
        print(f"question {p}, subsample=colsample=0.6")
        xgb_params_by_question[p]["params"]['colsample_bytree']= 0.6
        xgb_params_by_question[p]["params"]['subsample']= 0.6

question 14, subsample=colsample=0.6
question 15, subsample=colsample=0.6


In [16]:
QUESTIONS_NOT_TRAIN=[2,18]
from sklearn.model_selection import StratifiedKFold,KFold,cross_val_score,cross_val_predict

N_SPLITS=4

In [17]:
clf1 = XGBClassifier()
clf1.load_model(f'/kaggle/input/models-newfeatures/XGB_question{9}_model1_fold_{0}.xgb')
print(clf1.get_params())

{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.7, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'gamma': 0, 'gpu_id': -1, 'grow_policy': 'lossguide', 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.02, 'max_bin': 256, 'max_cat_to_onehot': 4, 'max_delta_step': 0, 'max_depth': 5, 'max_leaves': 0, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 699, 'n_jobs': 16, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 32, 'reg_alpha': 0, 'reg_lambda': 1, 'sampling_method': 'uniform', 'scale_pos_weight': 0.5, 'subsample': 0.8, 'tree_method': 'hist', 'validate_parameters': 1, 'verbosity': None}


In [18]:
import jo_wilder
env = jo_wilder.make_env()
iter_test = env.iter_test()

In [19]:
FEATURES_QUESTIONS={}
models={}
for t in range(1,19):
    if t not in QUESTIONS_NOT_TRAIN:
        models[t]=[]
        for kfold in range(N_SPLITS):
            
            clf1 = XGBClassifier()
            clf1.load_model(f'/kaggle/input/models-newfeatures/XGB_question{t}_model1_fold_{kfold}.xgb')
            clf2 = XGBClassifier()
            clf2.load_model(f'/kaggle/input/models-newfeatures/XGB_question{t}_model2_fold_{kfold}.xgb')
            models[t].append((clf1,clf2))
            print(clf1.get_params())
    
    if t not  in QUESTIONS_BASE_FEATURES:
            FEATURES_QUESTIONS[t]=[]
            for kfold in range(N_SPLITS):
                
                if t in QUESTION_NEW_FEATURES:
                            print(f"Question {t}, reading new features")
                            f1=pd.read_csv(f"/kaggle/input/features-dict/question_{t}_summary_f0.csv")
                            f2=pd.read_csv(f"input/features-dict/question_{t}_summary_f1.csv")
                elif t in QUESTION_NEW_FEATURES_FOLD:
                            print(f"Question {t}, reading new fold features")
                            f1=pd.read_csv(f"/kaggle/input/features-dict/question_{t}_fold_{kfold}_f0.csv")
                            f2=pd.read_csv(f"input/features-dict/question_{t}_fold_{kfold}_f1.csv")

                if t in QUESTION_NEW_FEATURES_RANDOM:
                            print("Using random features to cut")
                            prev_feat_1=len(f1)
                            prev_feat_2=len(f2)
                            f1=list(f1.loc[(f1.importance>f1.loc[f1.column.str.contains("random")].importance.max())&(f1.importance>0)].column.values)
                            f2=list(f2.loc[(f2.importance>f2.loc[f2.column.str.contains("random")].importance.max())&(f2.importance>0)].column.values)
                            f1=[e for e in f1 if not "random" in e]
                            f2=[e for e in f2 if not "random" in e]
                            print(f"Numero de features: {len(f1)}, {len(f2)} (antes {prev_feat_1}, { prev_feat_2})")


                else:
                            prev_feat_1=len(f1)
                            prev_feat_2=len(f2)
                            f1=list(f1.loc[f1.importance>0].column.values)
                            f2=list(f2.loc[f2.importance>0].column.values)
                            f1=[e for e in f1 if not "random" in e]
                            f2=[e for e in f2 if not "random" in e]
                            print(f"Numero de features: {len(f1)}, {len(f2)} (antes {prev_feat_1}, { prev_feat_2})")
                            
                FEATURES_QUESTIONS[t].append((f1,f2))
        


{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.7, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'gamma': 0, 'gpu_id': -1, 'grow_policy': 'lossguide', 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.022, 'max_bin': 256, 'max_cat_to_onehot': 4, 'max_delta_step': 0, 'max_depth': 6, 'max_leaves': 0, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 800, 'n_jobs': 16, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 32, 'reg_alpha': 0, 'reg_lambda': 1, 'sampling_method': 'uniform', 'scale_pos_weight': 0.39999999999999997, 'subsample': 0.8, 'tree_method': 'hist', 'validate_parameters': 1, 'verbosity': None}
{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel':

In [20]:
limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}

count = 0
test_prev=None
for (test_new, sample_submission) in iter_test:
        test_new=test_new.sort_values(by="elapsed_time")
        sessions=[int(e.split("_")[0]) for e in list(sample_submission.loc[:,"session_id"].unique())]   
        session_id = test_new.session_id.values[0]
        grp = test_new.level_group.values[0]
        a,b = limits[grp]
        test_new = (pl.from_pandas(test_new)
                  .drop(["fullscreen", "hq", "music"])
                  .with_columns(columns))
        if test_prev is  None:
            test_prev=test_new
        else:
            test_prev=pl.concat([test_prev,test_new])
        del test_new
        gc.collect()
        test = feature_engineer_pl(test_prev.filter(pl.col('session_id').is_in(sessions)), grp, use_extra=False, feature_suffix='')
        test=test.set_index("session_id")
        
        # ------------------- level 0-4 ---------------------------------
        
    
        for t in range(a,b):
            #print(t)
            mask = sample_submission.session_id.str.contains(f'q{t}')
            sessions=[int(e.split("_")[0]) for e in list(sample_submission.loc[mask,"session_id"].values)]
            if t not in QUESTIONS_NOT_TRAIN:
                FEATURES = features_use_because_of_importances[t]
                if t in [3,12,13]:
                    FEATURES=list(set(FEATURES).intersection(set(test.columns)))
                    FEATURES.sort()
                    #print(f"question {t}, sorting features ")
                p=None
                for kfold in range(N_SPLITS):

                    if t in QUESTIONS_BASE_FEATURES:
                        #print(f"Question {t}, using base features")
                        f1=[]
                        f2=[]
                        for i,f in enumerate(FEATURES):
                                if i%2:
                                    f1.append(f)
                                else:
                                    f2.append(f)
                    
                    else:
                        f1,f2=  FEATURES_QUESTIONS[t][kfold]
                    


                    clf1=models[t][kfold][0]
                    clf2=models[t][kfold][1]
                    p_aux = clf1.predict_proba(test.loc[sessions,f1].astype('float32'))[:,1]+clf2.predict_proba(test.loc[sessions,f2].astype('float32'))[:,1]
                    p_aux=p_aux/2
                    if p is None:
                        p=p_aux
                    else:
                        p=p+p_aux

                p=p/N_SPLITS
      
                P= xgb_params_by_question[t]["umbral"]
                sample_submission.loc[mask,'correct'] = (p>P).astype(int)
            elif t in [2,3,18,12]:
                  
          
                sample_submission.loc[mask,'correct'] = 1
            elif t==13:
                sample_submission.loc[mask,'correct'] = 0
                
                
        env.predict(sample_submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [21]:
pd.set_option("display.max_rows",200)
pd.read_csv('submission.csv')

,session_id,correct
0,20090109393214576_q1,0
1,20090109393214576_q2,1
2,20090109393214576_q3,1
3,20090109393214576_q4,1
4,20090109393214576_q5,0
5,20090109393214576_q6,1
6,20090109393214576_q7,1
7,20090109393214576_q8,0
8,20090109393214576_q9,1
9,20090109393214576_q10,0
